In [1]:
import pandas as pd
import numpy as np

### 1. Cleaning teams matches stats

This code reads in two CSV files, 'posesion.csv' and 'shot.csv', and processes the data to create a single dataframe of team data for the Premier League. It performs the following steps:

* Reads in the 'posesion.csv' file and selects only the rows with 'Comp' equal to 'Premier League'.
* Reads in the 'shot.csv' file and selects only the rows with 'Comp' equal to 'Premier League'.
* Finds the intersection of the columns in the two dataframes and performs an inner merge on the dataframes using these columns.
* Selects a specific set of columns from the merged dataframe and renames some of the columns to avoid potential issues.
* Fills any missing values with 0 and converts the 'Fecha' column to a datetime data type.
* Converts the 'GF' and 'GC' columns to integer data types.
* Maps the values in the 'Resultado' column to a new 'pts' column using a dictionary.
* Remaps the team names in the 'Equipo' and 'Adversario' columns to their correct names using a dictionary.
* Sorts the dataframe by the 'Equipo' and 'Fecha' columns in ascending order.
* Selects specific columns from the dataframe.
* Writes the dataframe to a CSV file called 'teams.csv'.

Finally, the modified dataframe is returned and printed to the console.

In [2]:
# Import pandas library
import pandas as pd

# Read in 'posesion.csv' file, setting the first column as the index
pos = pd.read_csv('posesion.csv', index_col = 0)

# Filter the dataframe to only include rows where the 'Comp' column is equal to 'Premier League'
pos = pos.query("Comp == 'Premier League'")

# Read in 'shot.csv' file, setting the first column as the index
shoot = pd.read_csv('shot.csv', index_col = 0)

# Filter the dataframe to only include rows where the 'Comp' column is equal to 'Premier League'
shoot = shoot.query("Comp == 'Premier League'")

# Find the intersection of the columns in the two dataframes and store it in a list called "match_cols"
match_cols = list(set(pos.columns).intersection(set(shoot.columns)))

# Merge the two dataframes on the common columns, storing the result in a new dataframe called "teams"
teams = shoot.merge(pos, on = match_cols, how = 'inner')

# Assert that the number of rows in "teams" is equal to the number of rows in "pos"
assert len(teams) == len(pos)

# Select specific columns from the "teams" dataframe and store them in a new dataframe
sel_cols = ['year','Fecha','Equipo','Adversario','Sedes', 'Resultado', 'GF', 'GC', 'Dis', 'DaP', '% de TT', 'G/T', 'G/TalArc',
       'Dist','Pos.' ,'FK', 'TP', 'TPint', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG']
teams = teams[sel_cols]

# Replace any missing values with 0
teams = teams.fillna(0)

# Convert the 'Fecha' column to datetime format
teams['Fecha']= pd.to_datetime(teams['Fecha'].astype('str'))

# Convert the 'GF' and 'GC' columns to integer format
teams['GF'] = teams.GF.astype(np.int32)
teams['GC'] = teams.GC.astype(np.int32)

# Create a dictionary to map result strings to numerical values
res_map = {'V': 3, 'D': 0, 'E':1}

# Map the values in the 'Resultado' column using the dictionary and store the result in a new 'pts' column
teams['pts'] = teams['Resultado'].map(res_map)

# Find the unique values in the 'Equipo' and 'Adversario' columns and store them in separate sets
equipo = set(teams.Equipo.unique())
adv = set(teams.Adversario.unique())

# Find the intersection of the two sets
inter = equipo.intersection(adv)

# Find the difference between the two sets
differ = equipo.difference(adv)

# Find the union of the two sets
all_names = equipo.union(adv)

# Find the difference between the union and the difference
team_names = all_names.difference(differ)

# Import the 'process' module from the 'thefuzz' library
from thefuzz import process

# For each team name in the 'differ' set, print the closest matching team name from the 'team_names' set
for name in differ:
    print(name + ': ',process.extract(name, team_names, limit = 1)[0][0])



Tottenham Hotspur:  Tottenham
Sheffield United:  Sheffield Utd
Newcastle United:  Newcastle Utd
West Bromwich Albion:  West Brom
Manchester United:  Manchester Utd
Nottingham Forest:  Nott'ham Forest
West Ham United:  West Ham
Brighton and Hove Albion:  Brighton
Wolverhampton Wanderers:  Wolves


C:\Users\DELL\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Create a dictionary to map team names to their correct names
team_names_map = {'Wolverhampton Wanderers':  "Wolves",
'Sheffield United':  "Sheffield Utd",
'Nottingham Forest':  "Nott'ham Forest",
'Brighton and Hove Albion':  "Brighton",
'Newcastle United':  "Newcastle Utd",
'West Ham United':  "West Ham",
'Tottenham Hotspur':  "Tottenham",
'Manchester United':  "Manchester Utd",
'West Bromwich Albion':  'West Brom'}

# Use the 'replace' method to map the team names in the 'Equipo' column to their correct names
teams.Equipo = teams['Equipo'].replace(team_names_map)

# Use the 'replace' method to map the team names in the 'Adversario' column to their correct names
teams.Adversario = teams.Adversario.replace(team_names_map)

# Sort the 'teams' dataframe by the 'Equipo' and 'Fecha' columns in ascending order
teams = teams.sort_values(['Equipo','Fecha'], ascending = [True, True])

# Rename some of the columns to avoid potential issues
teams.columns = ['year', 'date', 'team', 'opponent', 'sede', 'resultado', 'gf', 'gc','dis','dap','po_TT','g_e_t','g_e_dap','dist','posse','fk','tp','tp_int','xg','npxg','npxg_e_dis','g_m_xg','npg_m_npxg','pts']

# Select specific columns from the 'teams' dataframe and store them in a new dataframe
teams = teams[['year', 'date', 'team', 'opponent', 'sede', 'resultado', 'pts','gf', 'gc','dis','dap','po_TT','g_e_t','g_e_dap','dist','posse','fk','tp','tp_int','xg','npxg','npxg_e_dis','g_m_xg','npg_m_npxg']]
teams = teams.reset_index(drop = True)
# Write the 'teams' dataframe to a CSV file called 'teams.csv'
teams.to_csv('teams.csv')

# Print the 'teams' dataframe
teams


,year,date,team,opponent,sede,resultado,pts,gf,gc,dis,...,dist,posse,fk,tp,tp_int,xg,npxg,npxg_e_dis,g_m_xg,npg_m_npxg
0,2021,2020-09-12,Arsenal,Fulham,Visitante,V,3,3,0,13.0,...,14.1,54.0,2.0,0.0,0.0,1.9,1.9,0.20,1.1,1.1
1,2021,2020-09-19,Arsenal,West Ham,Local,V,3,2,1,7.0,...,14.8,62.0,0.0,0.0,0.0,1.1,1.1,0.16,0.9,0.9
2,2021,2020-09-28,Arsenal,Liverpool,Visitante,D,0,1,3,4.0,...,15.0,34.0,0.0,0.0,0.0,1.3,1.3,0.32,-0.3,-0.3
3,2021,2020-10-04,Arsenal,Sheffield Utd,Local,V,3,2,1,6.0,...,16.3,64.0,0.0,0.0,0.0,0.6,0.6,0.09,1.4,1.4
4,2021,2020-10-17,Arsenal,Manchester City,Visitante,D,0,0,1,11.0,...,18.4,42.0,2.0,0.0,0.0,0.9,0.9,0.08,-0.9,-0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,2023,2022-10-18,Wolves,Crystal Palace,Visitante,D,0,1,2,10.0,...,16.3,49.0,1.0,0.0,0.0,0.8,0.8,0.08,0.2,0.2
1808,2023,2022-10-23,Wolves,Leicester City,Local,D,0,0,4,21.0,...,15.7,60.0,1.0,0.0,0.0,2.1,2.1,0.10,-2.1,-2.1
1809,2023,2022-10-29,Wolves,Brentford,Visitante,E,1,1,1,13.0,...,20.4,42.0,0.0,0.0,0.0,0.6,0.6,0.05,0.4,0.4
1810,2023,2022-11-05,Wolves,Brighton,Local,D,0,2,3,7.0,...,18.3,29.0,0.0,1.0,1.0,1.2,0.4,0.06,0.8,0.6


### 2. Cleaning lineups database

This code reads in a CSV file called 'alineaciones.csv' and processes the data to create a dataframe of lineup data for soccer matches. It performs the following steps:

* Reads in the 'alineaciones.csv' file and selects only the rows with 'num' not equal to 'Banquillo'.
* Creates a dictionary that maps the Spanish month names to their corresponding month numbers.
* Defines a function called 'convert_date_format' that takes in a date string in the format 'day-month-year' and returns it in the format 'day-month number-year'.
* Applies the 'convert_date_format' function to the 'fecha' column and stores the result in a new 'date' column.
* Converts the 'date' column to a datetime data type using the '%d-%m-%Y' format string.
* Selects specific columns from the dataframe and renames some of the columns.
* Sorts the dataframe by the 'date' and 'team' columns in ascending order.
* Writes the dataframe to a CSV file called 'lineups.csv'.

Finally, the modified dataframe is returned and printed to the console.

In [4]:
# Read in the 'alineaciones.csv' file and select only the rows where 'num' is not equal to 'Banquillo'
lineups = pd.read_csv('alineaciones.csv', index_col = 0)
lineups = lineups[lineups.num != 'Banquillo']

# Create a dictionary that maps the Spanish month names to their corresponding month numbers
month_dict = {
    'Enero': '01',
    'Febrero': '02',
    'Marzo': '03',
    'Abril': '04',
    'Mayo': '05',
    'Junio': '06',
    'Julio': '07',
    'Agosto': '08',
    'Septiembre': '09',
    'Octubre': '10',
    'Noviembre': '11',
    'Diciembre': '12'
}

def convert_date_format(date_string):
    """
    Convert a date string from 'day-month-year' to 'day-month number-year' format.
    
    Parameters:
    date_string (str): The date string to convert.
    
    Returns:
    str: The converted date string.
    """
    day, month, year = date_string.split('-')
    month_number = month_dict[month]
    return f"{day}-{month_number}-{year}"

# Apply the 'convert_date_format' function to the 'fecha' column and store the result in a new 'date' column
lineups['date'] = lineups['fecha'].apply(convert_date_format)

# Convert the 'date' column to a datetime data type using the '%d-%m-%Y' format string
lineups['date'] = pd.to_datetime(lineups['date'], format = "%d-%m-%Y")

# Select specific columns from the dataframe and rename some of the columns
lineups = lineups[['date','equipo' ,'Nombre', 'tit']]
lineups.columns = ['date', 'team', 'name', 'start']

# Sort the dataframe by the 'date' and 'team' columns in ascending order
lineups = lineups.sort_values(['date', 'team'], ascending = [True, True])
lineups = lineups.reset_index(drop = True)
# Write the dataframe to a CSV file called 'lineups.csv'
lineups.to_csv('lineups.csv')

# Print the modified dataframe
lineups


,date,team,name,start
0,2020-09-12,Arsenal,Bernd Leno,1
1,2020-09-12,Arsenal,Héctor Bellerín,1
2,2020-09-12,Arsenal,Kieran Tierney,1
3,2020-09-12,Arsenal,Gabriel Dos Santos,1
4,2020-09-12,Arsenal,Alexandre Lacazette,1
...,...,...,...,...
35917,2022-12-26,Wolves,Toti Gomes,0
35918,2022-12-26,Wolves,Connor Ronan,0
35919,2022-12-26,Wolves,Matheus Nunes,0
35920,2022-12-26,Wolves,Adama Traoré,0


### 3. Cleaning players' stats 

This next code performs the following tasks:

* Reads the CSV files 'players_2021.csv', 'players_2022.csv', and 'players_2023.csv' and stores them in the dataframes 'players21', 'players22', and 'players23', respectively.

* Concatenates the dataframes 'players23', 'players22', and 'players21' into a single dataframe 'players'.

* Filters the rows in the dataframe 'players' to only include those from teams that are in the list 'all_names'.

* Filters the rows in the dataframe 'players' to exclude those that have a value of 'Estuvo en el día del partido, pero no jugó.' in the 'Mín' column.

* Removes all rows with missing values from the dataframe 'players'.

* Selects the columns of interest from the dataframe 'players' and stores them in the variable 'sel_cols'.

* Renames the columns in the dataframe 'players' to more meaningful names.

* Converts the values in the numeric columns of the dataframe 'players' to float format.

* Converts the 'date' column in the dataframe 'players' to datetime format.

* Sorts the dataframe 'players' by the 'name' and 'date' columns in ascending order.

* Replaces the team names in the 'team' column of the dataframe 'players' using the 'team_names_map' dictionary.

* Resets the index of the dataframe 'players' and drops the old index.

* Saves the dataframe 'players' to a CSV file named 'players.csv'.

In [5]:
# Read in the player data from the CSV files for the 2021, 2022, and 2023 seasons
players21 = pd.read_csv('players_2021.csv', index_col = 0)
players22 = pd.read_csv('players_2022.csv', index_col = 0)
players23 = pd.read_csv('players_2023.csv', index_col = 0)

# Concatenate the dataframes for the three seasons into a single dataframe called 'players'
players = pd.concat([players23,players22,players21], ignore_index = True)

# Select only the rows in 'players' where the 'Equipo' column is in the 'all_names' set
players = players[players['Equipo'].isin(all_names)]

# Remove rows where the 'Mín' column is equal to "Estuvo en el día del partido, pero no jugó."
players = players[players['Mín'] != 'Estuvo en el día del partido, pero no jugó.']

# Remove rows with missing values
players = players.dropna()

# Select specific columns from the dataframe and rename some of the columns
players = players[['Nombre','Fecha','Equipo','Adversario' ,'Mín', 'Gls.', 'Ass', 'TP', 'TPint',
       'Dis', 'DaP', 'TA', 'TR', 'Toques', 'Tkl', 'Int', 'Bloqueos', 'xG',
       'npxG', 'xAG', 'ACT', 'ACG', 'Cmp', 'Int.', '% Cmp', 'Prog',
       'Con éxito', 'Int..1']]

players.columns = ['name','date', 'team', 'opponent', 'min','g', 'ass', 'tp', 'tpint', 
                   'dis', 'dap', 'ta', 'tr', 'toques','tkl','interceps','blocks','xg',
                   'npxg','xag', 'act','acg', 'cmp', 'int_pass', 'po_cmp','prog', 
                   'reg_success', 'reg_int']

# Convert the following columns to float type
num_cols = ['min','g', 'ass', 'tp', 'tpint', 
                   'dis', 'dap', 'ta', 'tr', 'toques','tkl','interceps','blocks','xg',
                   'npxg','xag', 'act','acg', 'cmp', 'int_pass', 'po_cmp','prog', 
                   'reg_success', 'reg_int']
for col in num_cols:
    players[col] = players[col].astype('float')
    
# Convert the 'date' column to datetime type
players.date = pd.to_datetime(players.date)

# Sort the dataframe by 'name' and 'date' columns in ascending order
players = players.sort_values(['name','date'], ascending = [True, True])

# Replace the values in the 'team' column with the corresponding values from the 'team_names_map' dictionary
players['team'] = players['team'].replace(team_names_map)

# Reset the index of the dataframe
players = players.reset_index(drop = True)


# Save the modified dataframe to a CSV file
players.to_csv('players.csv')

# View the modified dataframe

players

,name,date,team,opponent,min,g,ass,tp,tpint,dis,...,npxg,xag,act,acg,cmp,int_pass,po_cmp,prog,reg_success,reg_int
0,Aaron Connolly,2020-09-14,Brighton,Chelsea,46.0,0.0,0.0,0.0,0.0,3.0,...,0.2,0.0,1.0,0.0,2.0,2.0,100.0,1.0,0.0,2.0
1,Aaron Connolly,2020-09-20,Brighton,Newcastle Utd,89.0,1.0,0.0,0.0,0.0,2.0,...,0.1,0.0,1.0,0.0,10.0,13.0,76.9,2.0,1.0,2.0
2,Aaron Connolly,2020-09-26,Brighton,Manchester Utd,74.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,6.0,7.0,85.7,0.0,0.0,0.0
3,Aaron Connolly,2020-10-03,Brighton,Everton,65.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,5.0,20.0,0.0,0.0,1.0
4,Aaron Connolly,2020-10-18,Brighton,Crystal Palace,13.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,50.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25753,Yves Bissouma,2022-11-06,Tottenham,Liverpool,87.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,2.0,0.0,51.0,57.0,89.5,4.0,2.0,2.0
25754,Yves Bissouma,2022-11-12,Tottenham,Leeds United,22.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,18.0,18.0,100.0,1.0,1.0,1.0
25755,Yves Bissouma,2022-12-26,Tottenham,Brentford,90.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,54.0,61.0,88.5,2.0,0.0,0.0
25756,Zack Steffen,2021-01-03,Manchester City,Chelsea,90.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,30.0,83.3,0.0,0.0,0.0
